# Slack Sentiment Analysis

This notebook is a somewhat pointless but simple exercise where we use [Natural Language Toolkit](https://www.nltk.org) models, both pre-trained and trained by us, to figure out if the general sentiment in a slack channel is positive or negative assuming (most likely incorrectly) that we can judge such a thing using amazon, yelp and movie reviews as examples of positive or negative chat.

## Fetch Slack Data


>If you do not want to use a slack channel you don't have to. If you skip ahead to **"Prepare the Data"** you will find some code where you can comment out a bit to use sample data provided by NLTK instead.

To get messages from a slack channel you will need a token that has permissions to read from that channel as well as the ID of the channel. 

**To get the slack channel ID:**

There is probably an easier way but the one I use is to right click the channel name in the channel menu and select "Copy link". The link will look something like <https://your-org.slack.com/archives/C12345>. The channel ID is the last piece. That's C12345 in the example URL. 

**To get a token:**

Setup a bot user following these instructions <https://api.slack.com/bot-users>. 

The OAuth scope must allow `channels:history`. Add the bot user to the channel you wish to read messages from. The channel should be public. I have not tested any of this with private channels. 

The next section assumed you have 2 environment variables:

`SLACK_SENTIMENT_BOT_TOKEN` - The bot token we setup previously.

`SLACK_SENTIMENT_CHANNEL_ID` - The channel ID we retrieved previously.

If you are not familiar with how to setup environment variables , feel free to just replace those values with hardcoded values but obviously be careful not to accidentally push your token up to Github or something like that :)

Incidentally, I do have a post on managing secrets while developing if you're interested: [a post on managing secrets](https://dev.to/ruarfff/managing-local-app-secrets-and-sharing-secrets-with-your-team-34m1).

In [ ]:
import os
# Import WebClient from Python SDK (github.com/slackapi/python-slack-sdk)
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

token = os.environ.get("SLACK_SENTIMENT_BOT_TOKEN")
channel_id = os.environ.get("SLACK_SENTIMENT_CHANNEL_ID")

client = WebClient(token=token)

conversation_history = []


try:
    result = client.conversations_history(channel=channel_id)
    conversation_history = result["messages"]
except SlackApiError as e:
    print("Error getting conversations: {}".format(e))

## Prepare the Data

We will try a few things out here. We will split out the data into sentences and words using NLTK tokenize methods. We will clean up the data to remove a large amount of noise. 

There are a lof ot pieces in the text that aren't really useful. Punctuation for example. Also things like emojis and various words that don't hold much meaning out of context (NLTK provdes a set of these called "stopwords"). It would actually be useful to include emojis in this kind of analysis but let's keep it simple and exclude them here. 

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import itertools

# If you do not want to use a slack channel, uncomment this and remove/comment out the next get_test_data function
# def get_test_data():
#    download('state_union')
#    words = nltk.corpus.state_union.words()
#    sentences = sent_tokenize(' '.join(words))
#    return (sentences, words)


def get_test_data():
    all_text = [message['text'] for message in conversation_history if 'text' in message]
    tokenized_text = [sent_tokenize(text) for text in all_text]
    sentences = list(itertools.chain(*tokenized_text))
    tokenized_words = [word_tokenize(sentence) for sentence in sentences]
    words = list(itertools.chain(*tokenized_words))
    return (sentences, words)

main_sentences, main_words = get_test_data()

## Use a Pre-trained Model

Let's look at a fairly simple method to take the data we just prepared and run it through a pre trained model. 

Later we will look at using all the words from the slack chat with a mixture of frequency distribution and naive bayes to gauge the genral sentiment with a model we train. Here we will instead try going over the chat sentence by sentnece and create a pie chart of Positive VS Negative.

In [ ]:
from nltk import download
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer

# The SentimentIntensityAnalyzer model needs us to pull down the vader_lexicon
download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

num_pos = 0
num_neg = 0

def is_positive(sentence: str) -> bool:
    return sia.polarity_scores(sentence)["compound"] > 0

for s in main_sentences:
    if is_positive(s):
        num_pos +=1
    else:
        num_neg += 1
        
labels = 'Positive', 'Negative'
sizes = [num_pos, num_neg]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')

plt.show()

## Clean the Data

From this point on we won't be using the sentences anymore and will focus on the individual words instead. Let's create a frequency distribution graph of the words. This is to demonstrate how many of them are more of less useless.

In [ ]:
from nltk.probability import FreqDist

fdist = FreqDist(main_words)

fdist.plot(30,cumulative=False)
plt.show()

When we start training our own model we will be using a method where we store each word along wiht its frequency and train a model using those features (the words and frequency) along with labels (positive and negative).

We will need 2 functions. One to clean out data, removing noise such as meaningless words, punctuation etc. that could introduce greater incuracy to our analysis. Then we need a function to convert a stream of words into a hashmap with words and number of occurences. 

In [ ]:
download('stopwords')
download('names')

from nltk.corpus import stopwords, names
from string import punctuation

name_words = set([n.lower() for n in names.words()])
stop_words = set(stopwords.words("english"))

def clean_words(words):
    return [w for w in [w.lower() for w in words if w.isalpha()] if w not in stop_words and w not in name_words and w not in punctuation]

def word_counts(words):
    counts = {}
    for word in words:
        counts[word] = counts.get(word, 0) + 1
    return counts

In [ ]:
main_words = clean_words(main_words)
main_words_counts = word_counts(main_words)

Let's see what effect that had.

In [ ]:
fdist = FreqDist(main_words)
fdist.plot(30,cumulative=False)
plt.show()

We will come back to those values later. First we will work on training a model from existing labelled data sets.

## Training Our Own Model

Using a pre-trained model works to some degree but isn't very interesting. Let's look at training our own model and using it to classify the data we prepared earlier. 

We are going to train a model using various data sets with text that is labelled as positive or negative. We are going to do some processing on this data to split it up into words with labels. This is called extracting features.

We are going to take some datasets from a couple of places and put them all together to create our features.

First let's read some data stored in this repository which was downloaded from <https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences>. These files contain text with a label `0` for negative and `1` for positive.

In [ ]:
import pandas as pd

amazon = pd.read_csv('sentiment_labelled_sentences/amazon_cells_labelled.txt', names=['review', 'sentiment'], sep='\t') 
imdb = pd.read_csv('sentiment_labelled_sentences/imdb_labelled.txt', names=['review', 'sentiment'], sep='\t') 
yelp = pd.read_csv('sentiment_labelled_sentences/yelp_labelled.txt', names=['review', 'sentiment'], sep='\t') 

NLTK also provides some labelled data in its corpus. Let's grab that too for good measure.

In [ ]:
download('movie_reviews')

### Split the Data

Let's process all the data into 2 collections of positive and negative. We use 2 helper functions that we created earlier. One to clean up the data set. The other to create the hashmap of words with their number of occurrences. 

In [ ]:
reviews = [*amazon['review'].values, *imdb['review'].values, *yelp['review'].values]
sentiment = [*amazon['sentiment'].values, *imdb['sentiment'].values, *yelp['sentiment'].values]

positive_reviews = []
negative_reviews = []

from nltk.corpus import movie_reviews

# Process the corpus data which is split into files with IDs of pos and neg
for f in movie_reviews.fileids('pos'):
    positive_reviews.append((word_counts(clean_words(movie_reviews.words(f))), 'pos'))
for f in movie_reviews.fileids('neg'):
    negative_reviews.append((word_counts(clean_words(movie_reviews.words(f))), 'neg'))

# Process the data we extracted from files into reviews and sentiment lists with matching indexes  
for i, r in enumerate(reviews):
    review_words = word_tokenize(r)
    if sentiment[i] == 1:
        positive_reviews.append((word_counts(clean_words(review_words)), 'pos'))
    else:
        negative_reviews.append((word_counts(clean_words(review_words)), 'neg'))


Now we have all that training data, let's train a model.

We will use an 80/20 split which [may or may not be optimal here](https://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validation).

In [ ]:
from nltk.classify import NaiveBayesClassifier

# We will use 80% of the data set for training and 20% for testing
split_pct = .80

def split_set(review_set):
    split = int(len(review_set)*split_pct)
    return (review_set[:split], review_set[split:])

pos_train, pos_test = split_set(positive_reviews)
neg_train, neg_test = split_set(negative_reviews)

train_set = pos_train + neg_train
test_set = pos_test + neg_test

model = NaiveBayesClassifier.train(train_set)

In [ ]:
from nltk.classify.util import accuracy

# We can figure out how well the training went using ouer test set
print(100 * accuracy(model, test_set))

We can see ewhat our model thinks are the most informative features. The most informative featues in this case are words that appear far more frequently with the label `pos` or the label `neg`.

In [ ]:
model.show_most_informative_features()

Finally, let's see if our model thinks the general sentiment in the slack channel (or in whatever data set you used) is positive or negative.

In [ ]:
model.classify(main_words_counts)

## Using a Model for Real

Now we can save the model. You could push this model up somewhere and pull it in to some other useful location. For an example of how to deploy this model in a web service using docker please see the [Readme for this project](https://github.com/ruarfff/slack-sentiment/blob/main/README.md).

You can imagine some process whereby you train a model at some interval or on some event and push the model to some location that gets pulled down for use in some application or service.

In [ ]:
import pickle

model_file = open('.models/sentiment_classifier.pickle','wb')
pickle.dump(model, model_file)
model_file.close()

Just to show one way you might take that file and use it elsewhere.

In [ ]:
model_file = open('.models/sentiment_classifier.pickle', 'rb')
model = pickle.load(model_file)
model_file.close()

print(100 * accuracy(model, test_set))
model.show_most_informative_features()
model.classify(main_words_counts)